<a href="https://colab.research.google.com/github/ErikHartman/BMEN35/blob/2023/BMEN35_notebook_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise about xAI and omics

This exercise is about methods in the field of explainable AI (xAI) and they're application to omics data. Specifically, we'll use SHapley Additive exPlanations (SHAP) to introspect trained models to understand their reasoning during classification tasks.

The exercise has two parts:

1. In the first part, you'll get to train a neural network on the MNIST dataset (digits 0-9). Thereafter you'll explain a set of predictions using SHAP. 
2. In the second part, we'll apply a similar methodology but to a clinical proteomic dataset. Here, SHAP is used to explain what proteins a classifier deems important when classifying two different subtypes of septic acute kidney injury.




In [ ]:
!pip install shap
!pip install xgboost

## Part 1

Here you will train a convolutional neural network to classify the digits 0-9 (MINST dataset). Thereafter you will interpret the network using SHAP. Familiarize yourself with the code and make sure you understand every step.

In [ ]:
import numpy as np
import keras
from keras import layers
from keras.utils import to_categorical


num_classes = 10  # 0-9
input_shape = (28, 28, 1)  # pixels

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

batch_size = 128
epochs = 2

model = keras.Sequential(
    [
        layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("---- Model is trained, time for interpretation ----")

Now it's time to interpret using SHAP. 

The background is a representative set of datapoints which should capture the "average output" of the network. This is simply to establish a baseline that outputs can be compared against later when explaining the network.

In [ ]:
import shap

# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]

# Create the explainer object
e = shap.DeepExplainer(model, background)

# Compute the SHAP values
shap_values = e.shap_values(x_test[1:5])
# Show them in an image
shap.image_plot(shap_values, -x_test[1:5])

### Question: what features seem important for the different digits? You can change the x_test slice to see other digits.

## Part 2

Now it's time to apply the same principles but to some biological data.

The data used here is the septic AKI data (see lecture).

In [3]:
import pandas as pd

# load the data from GitHub
data_matrix = pd.read_csv(
    "https://raw.githubusercontent.com/ErikHartman/BMEN35/2023/data/quant_matrix.csv"
)
design_matrix = pd.read_csv(
    "https://raw.githubusercontent.com/ErikHartman/BMEN35/2023/data/inner_design_matrix.tsv",
    sep="\t",
)
human_proteome = pd.read_csv(
    "https://raw.githubusercontent.com/ErikHartman/BMEN35/2023/data/human_proteome.gz",
    sep=",",
)

In [4]:
def get_proteins_triv_name(proteins, proteome):
    # function to map Uniprot IDs to more common names.
    proteome["accession"] = proteome["accession"].apply(lambda x: x.split("_")[0])
    names = []
    for protein in proteins:
        if protein in proteome["accession"].values:
            m = proteome.loc[proteome["accession"] == protein]["trivname"].values
            assert len(m) == 1
            m = m[0].split("_")[0]
        else:
            m = protein
        names.append(m)
    return names

In [ ]:
group_one_samples = design_matrix[design_matrix["group"] == 1]["sample"].values
group_two_samples = design_matrix[design_matrix["group"] == 2]["sample"].values
data_matrix.head()

Preprocess the data.

In [ ]:
from sklearn import preprocessing

protein_labels = get_proteins_triv_name(
    data_matrix["Protein"].values, human_proteome
)  # the UniProt IDs are translated to more common names.

df1 = data_matrix[group_one_samples].T
df2 = data_matrix[group_two_samples].T
df1.columns = protein_labels
df2.columns = protein_labels

y = np.array(
    [0 for _ in group_one_samples] + [1 for _ in group_two_samples]
)  
df_X = pd.concat([df1, df2]).fillna(0)
X = df_X.to_numpy()
scaler = preprocessing.StandardScaler().fit(X)  # makes mean = 0 and variance = 1
X_scaled = scaler.transform(X)  # this is our scaled X

df_X_scaled = pd.DataFrame(X_scaled)  # puts everything into a pandas DataFrame
df_X_scaled.columns = df_X.columns
df_X_scaled

In [1]:
from xgboost.sklearn import XGBClassifier

""" Train and test how accuarge the XGBoost model is on the data here.
Import the necessary packages. Scikit-learn is available by default in Google Collab
so there is no need to !pip install if you're using that package. """

" Train and test how accuarge the XGBoost model is on the data here.\nImport the necessary packages. Sci-kit learn is available by default in Google Collab\nso there is no need to !pip install if you're using that package. "

In [2]:
""" Create some SHAP plots here! Feel free to stylize them and do whatever you'd like. """

" Create some SHAP plots here! Feel free to stylize them and do whatever you'd like"

### The last task is to look at your top scoring proteins. Can you find any reference or other information linking these to septic AKI? Write in markdown below.